## BloomCast – Search Engine Index
This section implements the search index described in the Word document:
- Pandas DataFrame index
- Stop words filtering
- Lemmatization
- RAG-based result presentation


In [192]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("punkt")
nltk.download("punkt_tab")   # חשוב! בהרבה Colab זה מה שחסר וגורם ל-LookupError
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [193]:
documents = {
    1: "Plant disease detection using image analysis and environmental sensors.",
    2: "Water stress in plants caused by low soil moisture and high temperature.",
    3: "Early pest detection in crops using AI and leaf image processing.",
    4: "Smart irrigation systems improve plant health and reduce water stress.",
    5: "Monitoring humidity and temperature helps prevent plant diseases."
}


In [194]:
custom_stop_words = {
    "the", "and", "is", "are", "of", "to", "in", "on", "for", "with",
    "data", "value", "values", "sensor", "sensors",
    "system", "systems", "from", "using", "used", "based"
}

stop_words = set(stopwords.words("english")).union(custom_stop_words)


In [195]:
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words
    ]
    return tokens


In [196]:
index = {}

for doc_id, text in documents.items():
    tokens = preprocess_text(text)
    for term in tokens:
        if term not in index:
            index[term] = set()
        index[term].add(doc_id)


In [197]:
index_df = pd.DataFrame([
    {"term": term, "DocIDs": sorted(list(doc_ids))}
    for term, doc_ids in index.items()
])

index_df


term        DocIDs
0           plant  [1, 2, 4, 5]
1         disease        [1, 5]
2       detection        [1, 3]
3           image        [1, 3]
4        analysis           [1]
5   environmental           [1]
6           water        [2, 4]
7          stress        [2, 4]
8          caused           [2]
9             low           [2]
10           soil           [2]
11       moisture           [2]
12           high           [2]
13    temperature        [2, 5]
14          early           [3]
15           pest           [3]
16           crop           [3]
17             ai           [3]
18           leaf           [3]
19     processing           [3]
20          smart           [4]
21     irrigation           [4]
22        improve           [4]
23         health           [4]
24         reduce           [4]
25     monitoring           [5]
26       humidity           [5]
27           help           [5]
28        prevent           [5]

In [198]:
def search(query):
    query_tokens = preprocess_text(query)
    results = set()

    for term in query_tokens:
        if term in index:
            results.update(index[term])

    return sorted(results)


In [199]:
search("plant disease and humidity")


[1, 2, 4, 5]

In [200]:
def rag_search(query):
    doc_ids = search(query)

    if not doc_ids:
        return "No relevant documents found."

    response = "RAG Result:\n\n"
    for doc_id in doc_ids:
        response += f"- Document {doc_id}: {documents[doc_id]}\n"

    response += "\nAI Insight: Based on the retrieved articles, monitoring humidity, temperature, and early visual symptoms is critical for preventing plant diseases and water stress."

    return response


In [201]:
print(rag_search("plant disease water stress"))


RAG Result:

- Document 1: Plant disease detection using image analysis and environmental sensors.
- Document 2: Water stress in plants caused by low soil moisture and high temperature.
- Document 4: Smart irrigation systems improve plant health and reduce water stress.
- Document 5: Monitoring humidity and temperature helps prevent plant diseases.

AI Insight: Based on the retrieved articles, monitoring humidity, temperature, and early visual symptoms is critical for preventing plant diseases and water stress.


part 2


## BloomCast – Smart Plant Monitoring System

BloomCast is a dynamic web-based platform designed to support precision agriculture.
The system enables real-time monitoring, analysis, and visualization of plant health
using simulated IoT sensor data, AI-based diagnosis, and interactive dashboards.

### Core Capabilities:
- Upload and visualization of plant images
- Real-time IoT sensor data simulation (temperature, humidity, light)
- AI-based plant health diagnosis
- Visual dashboards with dynamic charts
- Smart alerts and recommendations
- Gamification elements to encourage user engagement

All system data is generated dynamically to simulate real-world agricultural environments.


In [191]:
from IPython.display import IFrame
import tempfile, os

html_code ="""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>BloomCast — Plant Health Monitor</title>

  <!-- Icons -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.5.0/css/all.min.css"/>

  <style>
    :root{
      --bg: #f2fbf7;
      --panel: #ffffff;
      --panel2: #f7fffb;
      --text: #0f172a;
      --muted: #64748b;
      --border: rgba(15, 23, 42, 0.08);
      --shadow: 0 18px 45px rgba(16, 24, 40, 0.10);
      --shadow2: 0 10px 28px rgba(16, 24, 40, 0.08);
      --green: #18b07b;
      --green2: #0ea86d;
      --orange: #f59e0b;
      --red: #ef4444;
      --blue: #3b82f6;
      --radius: 18px;
    }

    *{ box-sizing: border-box; }
    body{
      margin:0;
      font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial;
      background: radial-gradient(900px 500px at 30% 10%, #dbfff1 0%, transparent 60%),
                  radial-gradient(900px 600px at 80% 90%, #e8fff6 0%, transparent 55%),
                  var(--bg);
      color: var(--text);
    }

    .app{
      display:flex;
      min-height: 100vh;
    }

    /* Sidebar */
    .sidebar{
      width: 270px;
      background: rgba(255,255,255,0.75);
      backdrop-filter: blur(10px);
      border-right: 1px solid var(--border);
      padding: 22px 18px;
      display:flex;
      flex-direction: column;
      gap: 16px;
      position: sticky;
      top:0;
      height: 100vh;
    }

    .brand{
      display:flex;
      align-items:center;
      gap: 12px;
      padding: 8px 10px;
      border-radius: 14px;
    }
    .logo{
      width: 42px; height: 42px;
      border-radius: 14px;
      display:flex; align-items:center; justify-content:center;
      background: linear-gradient(135deg, #1bd49a, #18b07b);
      color:white;
      box-shadow: 0 10px 25px rgba(24,176,123,0.25);
    }
    .brand h1{ font-size: 16px; margin:0; line-height:1.1; }
    .brand p{ margin:0; font-size: 12px; color: var(--muted); }

    .nav{
      display:flex;
      flex-direction: column;
      gap: 8px;
      margin-top: 6px;
    }
    .nav button{
      width:100%;
      display:flex;
      align-items:center;
      gap: 10px;
      border: 1px solid transparent;
      background: transparent;
      color: var(--text);
      padding: 12px 12px;
      border-radius: 14px;
      cursor:pointer;
      font-weight: 600;
      transition: .2s;
    }
    .nav button i{ width: 18px; text-align:center; color: #0b8f67; }
    .nav button.active{
      background: rgba(24,176,123,0.10);
      border-color: rgba(24,176,123,0.25);
    }
    .nav button:hover{
      background: rgba(24,176,123,0.08);
    }

    .sysStatus{
      margin-top:auto;
      padding: 14px;
      border-radius: 16px;
      background: rgba(24,176,123,0.08);
      border: 1px solid rgba(24,176,123,0.18);
      color: #0b8f67;
    }
    .sysStatus .dot{
      display:inline-block; width: 10px; height: 10px;
      border-radius: 50%;
      background: #19c37d;
      margin-right: 8px;
      box-shadow: 0 0 0 6px rgba(25,195,125,0.12);
    }
    .sysStatus small{ color: var(--muted); display:block; margin-top: 4px; }

    /* Main */
    .main{
      flex: 1;
      padding: 28px 28px 40px;
    }

    .topbar{
      display:flex;
      justify-content: space-between;
      align-items: center;
      gap: 16px;
      margin-bottom: 18px;
    }

    .breadcrumbs{
      display:flex;
      gap: 10px;
      align-items:center;
      color: var(--muted);
      font-weight: 600;
      font-size: 13px;
    }
    .crumb{
      display:inline-flex;
      align-items:center;
      gap: 8px;
      padding: 8px 12px;
      border-radius: 999px;
      background: rgba(24,176,123,0.10);
      color: #0b8f67;
      border: 1px solid rgba(24,176,123,0.18);
    }

    .actions{
      display:flex;
      gap: 10px;
      align-items:center;
    }
    .pill{
      border: 1px solid var(--border);
      background: rgba(255,255,255,0.75);
      border-radius: 999px;
      padding: 10px 12px;
      cursor:pointer;
      box-shadow: var(--shadow2);
      display:flex; align-items:center; gap: 10px;
    }
    .pill .badge{
      width: 18px; height: 18px;
      border-radius: 999px;
      background: #ef4444;
      color:white;
      font-size: 12px;
      display:flex; align-items:center; justify-content:center;
      font-weight: 700;
    }
    .primaryBtn{
      background: linear-gradient(135deg, #1bd49a, #18b07b);
      color:white;
      border:none;
      padding: 11px 14px;
      border-radius: 12px;
      cursor:pointer;
      box-shadow: 0 18px 35px rgba(24,176,123,0.25);
      font-weight: 800;
      display:flex; gap: 10px; align-items:center;
    }

    .pageTitle h2{ font-size: 34px; margin: 2px 0 6px; letter-spacing: -0.7px; }
    .pageTitle p{ margin:0; color: var(--muted); font-weight: 500; }

    /* Layout blocks */
    .grid{
      display:grid;
      grid-template-columns: repeat(12, 1fr);
      gap: 18px;
      margin-top: 18px;
    }

    .card{
      background: rgba(255,255,255,0.85);
      border: 1px solid var(--border);
      border-radius: var(--radius);
      box-shadow: var(--shadow);
      padding: 18px;
    }

    .miniCards{
      grid-column: 1 / -1;
      display:grid;
      grid-template-columns: repeat(12, 1fr);
      gap: 18px;
    }
    .mini{ grid-column: span 3; padding: 18px; display:flex; justify-content: space-between; gap: 12px; }
    .mini .icon{
      width: 44px; height: 44px;
      border-radius: 14px;
      display:flex; align-items:center; justify-content:center;
      background: rgba(24,176,123,0.10);
      color: #0b8f67;
      border: 1px solid rgba(24,176,123,0.18);
    }
    .mini .val{ font-size: 28px; font-weight: 900; margin: 0; }
    .mini .lbl{ margin: 0; color: var(--muted); font-weight: 600; }
    .mini .delta{ font-size: 12px; color: #0b8f67; font-weight: 800; }

    /* Dashboard blocks */
    .trend{ grid-column: span 8; min-height: 340px; }
    .sideStats{ grid-column: span 4; display:flex; flex-direction: column; gap: 18px; }
    .statBox{ padding: 18px; border-radius: var(--radius); border: 1px solid var(--border); background: rgba(255,255,255,0.85); }
    .statBox h4{ margin: 0 0 6px; font-size: 14px; color: var(--muted); }
    .statBox .big{ font-size: 32px; font-weight: 900; margin: 0; }
    .bar{
      height: 10px;
      border-radius: 999px;
      background: rgba(15,23,42,0.08);
      overflow:hidden;
      margin-top: 10px;
    }
    .bar > div{
      height: 100%;
      width: 0%;
      border-radius: 999px;
      background: linear-gradient(90deg, #1bd49a, #18b07b);
      transition: width .6s ease;
    }
    .tag{
      display:inline-flex;
      align-items:center;
      gap: 8px;
      padding: 6px 10px;
      border-radius: 999px;
      font-weight: 800;
      font-size: 12px;
      margin-top: 10px;
      background: rgba(24,176,123,0.10);
      color: #0b8f67;
      border: 1px solid rgba(24,176,123,0.18);
    }
    .tag.warn{ background: rgba(245,158,11,0.12); border-color: rgba(245,158,11,0.25); color: #b45309; }
    .tag.bad{ background: rgba(239,68,68,0.10); border-color: rgba(239,68,68,0.22); color: #b91c1c; }

    .lowerRow{ grid-column: 1 / -1; display:grid; grid-template-columns: repeat(12, 1fr); gap: 18px; }
    .donutCard{ grid-column: span 4; min-height: 260px; }
    .envCard{ grid-column: span 4; min-height: 260px; }
    .activityCard{ grid-column: span 4; min-height: 260px; }

    .activityList{ list-style:none; padding:0; margin: 10px 0 0; display:flex; flex-direction: column; gap: 10px; }
    .activityList li{
      display:flex; gap: 10px; align-items:flex-start;
      padding: 10px 12px;
      border-radius: 14px;
      background: rgba(15,23,42,0.03);
      border: 1px solid rgba(15,23,42,0.06);
    }
    .activityList i{ color: #0b8f67; margin-top: 2px; }
    .activityList small{ color: var(--muted); display:block; margin-top: 2px; }

    .careBanner{
      grid-column: 1 / -1;
      padding: 18px 18px;
      border-radius: 18px;
      background: linear-gradient(90deg, #1bd49a, #18b07b);
      color:white;
      display:flex;
      align-items:center;
      justify-content: space-between;
      box-shadow: 0 18px 40px rgba(24,176,123,0.25);
    }
    .careBanner h3{ margin:0 0 4px; }
    .careBanner p{ margin:0; opacity: 0.9; font-weight: 600; }
    .careBanner .right{
      display:flex; align-items:center; gap: 12px;
    }
    .ghostBtn{
      background: rgba(255,255,255,0.18);
      border: 1px solid rgba(255,255,255,0.28);
      color:white;
      padding: 10px 14px;
      border-radius: 12px;
      cursor:pointer;
      font-weight: 900;
    }

    /* Upload */
    .uploadWrap{ grid-column: 1 / -1; }
    .uploadBox{
      border: 2px dashed rgba(15,23,42,0.12);
      background: rgba(15,23,42,0.02);
      border-radius: 18px;
      padding: 28px;
      text-align:center;
      min-height: 240px;
      display:flex; flex-direction: column; align-items:center; justify-content:center;
      gap: 10px;
    }
    .uploadBox .bigIcon{
      width: 70px; height: 70px;
      border-radius: 18px;
      display:flex; align-items:center; justify-content:center;
      background: rgba(15,23,42,0.04);
      border: 1px solid rgba(15,23,42,0.08);
      color: var(--muted);
      font-size: 26px;
    }
    .hintCards{ grid-column: 1 / -1; display:grid; grid-template-columns: repeat(12,1fr); gap: 18px; }
    .hint{ grid-column: span 4; }
    .hint h4{ margin: 0 0 6px; }
    .hint p{ margin: 0; color: var(--muted); font-weight: 600; }

    /* Sensor page */
    .sensorHeader{ grid-column: 1 / -1; display:flex; justify-content: space-between; align-items:center; gap: 12px;}
    .badgeConn{
      display:inline-flex; gap: 8px; align-items:center;
      padding: 8px 12px; border-radius: 999px;
      background: rgba(24,176,123,0.10);
      border: 1px solid rgba(24,176,123,0.18);
      color: #0b8f67;
      font-weight: 900;
    }
    .cards4{ grid-column: 1 / -1; display:grid; grid-template-columns: repeat(12,1fr); gap: 18px; }
    .gauge{ grid-column: span 3; text-align:center; padding: 18px; }
    .gauge h4{ margin: 4px 0 0; color: var(--muted); font-size: 13px; }
    .gauge .num{ font-size: 34px; font-weight: 950; margin: 8px 0; }
    .scale{ color: var(--muted); font-weight: 800; font-size: 12px; display:flex; justify-content: space-between; }

    .aqi{ grid-column: 1 / -1; }
    .aqiRow{ display:flex; gap: 16px; align-items:center; }
    .aqiScore{ font-size: 48px; font-weight: 950; color: #7c3aed; min-width: 90px;}
    .aqiBar{ flex: 1; height: 12px; border-radius: 999px; overflow:hidden; background: rgba(15,23,42,0.08); }
    .aqiFill{ height:100%; width: 0%; transition: width .6s ease; background: linear-gradient(90deg, #ef4444, #f59e0b, #22c55e); }
    .aqiMeta{ min-width: 140px; text-align:right; color: var(--muted); font-weight: 800; }

    /* Search page */
    .searchHeader{ grid-column: 1 / -1; text-align:center; padding: 8px 0 6px;}
    .searchHeader h2{ margin: 0; font-size: 34px; letter-spacing:-0.6px;}
    .searchHeader p{ margin: 8px 0 0; color: var(--muted); font-weight: 600; }
    .searchBox{ grid-column: 3 / 11; }
    .searchInput{
      width:100%;
      border: 1px solid var(--border);
      border-radius: 14px;
      padding: 14px 14px;
      background: rgba(255,255,255,0.9);
      box-shadow: var(--shadow2);
      font-weight: 700;
      outline:none;
    }
    .filters{
      display:flex; gap: 10px; align-items:center;
      margin-top: 12px;
      flex-wrap: wrap;
    }
    .chip{
      border: 1px solid var(--border);
      background: rgba(255,255,255,0.85);
      border-radius: 999px;
      padding: 8px 12px;
      cursor:pointer;
      font-weight: 800;
      color: var(--muted);
    }
    .chip.active{
      background: rgba(24,176,123,0.10);
      border-color: rgba(24,176,123,0.24);
      color: #0b8f67;
    }
    .popular{ grid-column: 3 / 11; display:flex; gap: 10px; flex-wrap: wrap; }
    .resultGrid{ grid-column: 3 / 11; display:grid; grid-template-columns: repeat(12,1fr); gap: 18px; margin-top: 6px;}
    .resCard{ grid-column: span 4; }
    .resCard h3{ margin: 10px 0 4px; }
    .resCard p{ margin: 0; color: var(--muted); font-weight: 700; }
    .badgeSeverity{
      display:inline-flex; align-items:center; gap: 8px;
      padding: 6px 10px;
      border-radius: 999px;
      font-size: 12px;
      font-weight: 900;
      border: 1px solid;
      margin-top: 10px;
    }
    .sevLow{ background: rgba(34,197,94,0.12); border-color: rgba(34,197,94,0.25); color:#15803d; }
    .sevMed{ background: rgba(245,158,11,0.14); border-color: rgba(245,158,11,0.26); color:#b45309; }
    .sevHigh{ background: rgba(239,68,68,0.10); border-color: rgba(239,68,68,0.24); color:#b91c1c; }

    /* Responsive tweaks */
    @media (max-width: 1100px){
      .mini{ grid-column: span 6; }
      .trend{ grid-column: span 12; }
      .sideStats{ grid-column: span 12; flex-direction: row; }
      .statBox{ flex:1; }
      .donutCard,.envCard,.activityCard{ grid-column: span 12; }
      .searchBox,.popular,.resultGrid{ grid-column: 1 / -1; }
      .resCard{ grid-column: span 6; }
      .gauge{ grid-column: span 6; }
      .sidebar{ position: fixed; left:-280px; }
    }
    @media (max-width: 650px){
      .mini{ grid-column: span 12; }
      .resCard{ grid-column: span 12; }
      .gauge{ grid-column: span 12; }
      .main{ padding: 18px; }
    }

    /* Utility */
    .row{ display:flex; justify-content: space-between; align-items:flex-start; gap: 12px; }
    .subtle{ color: var(--muted); font-weight: 700; }
    .hidden{ display:none !important; }
  </style>
</head>

<body>
<div class="app">

  <!-- Sidebar -->
  <aside class="sidebar">
    <div class="brand">
      <div class="logo"><i class="fa-solid fa-leaf"></i></div>
      <div>
        <h1>BloomCast</h1>
        <p>Health Monitor</p>
      </div>
    </div>

    <div class="nav">
      <button class="active" data-page="dashboard"><i class="fa-solid fa-gauge-high"></i> Dashboard</button>
      <button data-page="upload"><i class="fa-solid fa-cloud-arrow-up"></i> Upload Image</button>
      <button data-page="sensors"><i class="fa-solid fa-wifi"></i> Sensor Data</button>
      <button data-page="search"><i class="fa-solid fa-magnifying-glass"></i> Disease Search</button>
    </div>

    <div class="sysStatus">
      <div><span class="dot"></span><strong>System Status</strong></div>
      <small id="sysText">All sensors online</small>
    </div>
  </aside>

  <!-- Main -->
  <main class="main">

    <div class="topbar">
      <div class="breadcrumbs">
        <span class="crumb"><i class="fa-solid fa-chart-line"></i> <span id="crumbText">Dashboard</span></span>
      </div>
      <div class="actions">
        <button class="pill" id="notifBtn" title="Notifications">
          <i class="fa-regular fa-bell"></i>
          <span class="badge" id="notifCount">3</span>
        </button>
        <button class="primaryBtn" id="scheduleBtn"><i class="fa-regular fa-calendar"></i> Schedule Care</button>
        <button class="pill" id="refreshBtn" title="Refresh data"><i class="fa-solid fa-rotate"></i> Refresh</button>
      </div>
    </div>

    <!-- DASHBOARD PAGE -->
    <section id="page-dashboard">
      <div class="pageTitle">
        <h2>Plant Health Overview</h2>
        <p>Monitor all your plants in one place</p>
      </div>

      <div class="grid">
        <div class="miniCards">
          <div class="card mini">
            <div>
              <p class="val" id="kpiPlants">--</p>
              <p class="lbl">Total Plants</p>
            </div>
            <div class="icon"><i class="fa-solid fa-seedling"></i></div>
          </div>

          <div class="card mini">
            <div>
              <p class="val"><span id="kpiHealth">--</span>%</p>
              <p class="lbl">Avg. Health</p>
            </div>
            <div class="icon"><i class="fa-solid fa-heart-pulse"></i></div>
          </div>

          <div class="card mini">
            <div>
              <p class="val" id="kpiWater">--</p>
              <p class="lbl">Water Today</p>
            </div>
            <div class="icon"><i class="fa-solid fa-droplet"></i></div>
          </div>

          <div class="card mini">
            <div>
              <p class="val" id="kpiAlerts">--</p>
              <p class="lbl">Alerts</p>
            </div>
            <div class="icon"><i class="fa-solid fa-bell"></i></div>
          </div>
        </div>

        <div class="card trend">
          <div class="row">
            <div>
              <h3 style="margin:0 0 6px;">Weekly Health Trend</h3>
              <div class="subtle">Dynamic values update automatically</div>
            </div>
            <button class="pill" style="box-shadow:none;"><i class="fa-solid fa-ellipsis"></i></button>
          </div>
          <canvas id="trendChart" height="90"></canvas>
        </div>

        <div class="sideStats">
          <div class="statBox" style="background: rgba(24,176,123,0.08); border-color: rgba(24,176,123,0.20);">
            <h4>Overall Health Score</h4>
            <p class="big" style="color:#0b8f67;"><span id="overallScore">--</span><span style="font-size:14px;color:var(--muted);">/100</span></p>
            <div class="bar"><div id="barOverall"></div></div>
            <div class="tag" id="tagOverall"><i class="fa-solid fa-circle-check"></i> Excellent</div>
          </div>

          <div class="statBox" style="background: rgba(245,158,11,0.12); border-color: rgba(245,158,11,0.22);">
            <h4>Hydration Level</h4>
            <p class="big" style="color:#b45309;"><span id="hydrationScore">--</span><span style="font-size:14px;color:var(--muted);">/100</span></p>
            <div class="bar"><div id="barHydration" style="background: linear-gradient(90deg,#f59e0b,#fbbf24);"></div></div>
            <div class="tag warn" id="tagHydration"><i class="fa-solid fa-circle-exclamation"></i> Moderate</div>
          </div>

          <div class="statBox" style="background: rgba(24,176,123,0.08); border-color: rgba(24,176,123,0.20);">
            <h4>Light Exposure</h4>
            <p class="big" style="color:#0b8f67;"><span id="lightScore">--</span><span style="font-size:14px;color:var(--muted);">/100</span></p>
            <div class="bar"><div id="barLight"></div></div>
            <div class="tag" id="tagLight"><i class="fa-solid fa-circle-check"></i> Excellent</div>
          </div>
        </div>

        <div class="lowerRow">
          <div class="card donutCard">
            <div class="row">
              <div>
                <h3 style="margin:0;">Plant Status</h3>
                <div class="subtle">Healthy / Moderate / Needs Care</div>
              </div>
            </div>
            <canvas id="donutChart" height="160"></canvas>
          </div>

          <div class="card envCard">
            <div class="row">
              <div>
                <h3 style="margin:0;">Environment</h3>
                <div class="subtle">Moisture % & Temp °C</div>
              </div>
            </div>
            <canvas id="envChart" height="160"></canvas>
          </div>

          <div class="card activityCard">
            <div class="row">
              <div>
                <h3 style="margin:0;">Recent Activity</h3>
                <div class="subtle">Auto-generated updates</div>
              </div>
            </div>
            <ul class="activityList" id="activityList"></ul>
          </div>

          <div class="careBanner">
            <div>
              <h3>Next Care Schedule</h3>
              <p id="careText">3 plants need watering tomorrow morning</p>
            </div>
            <div class="right">
              <button class="ghostBtn" id="viewScheduleBtn">View Schedule</button>
            </div>
          </div>
        </div>
      </div>
    </section>

    <!-- UPLOAD PAGE -->
    <section id="page-upload" class="hidden">
      <div class="pageTitle">
        <h2>Upload Your Plant Photo</h2>
        <p>Take a clear photo of leaves or affected areas for accurate analysis</p>
      </div>

      <div class="grid">
        <div class="card uploadWrap">
          <div class="uploadBox" id="dropZone">
            <div class="bigIcon"><i class="fa-solid fa-arrow-up-from-bracket"></i></div>
            <h3 style="margin:0;">Drag & drop your plant photo</h3>
            <div class="subtle">or click to browse files</div>
            <small class="subtle">Supports JPG, PNG, WEBP up to 10MB</small>
            <input type="file" id="fileInput" accept="image/*" style="display:none;">
            <img id="previewImg" src="" alt="" style="max-width:480px; width:100%; border-radius:18px; display:none; margin-top:14px; border:1px solid var(--border);"/>
            <div id="analysisBox" class="card hidden" style="width:100%; max-width: 720px; text-align:left; margin-top:14px; box-shadow:none;"></div>
          </div>
        </div>

        <div class="hintCards">
          <div class="card hint">
            <h4>🌤 Good Lighting</h4>
            <p>Use natural daylight for best results</p>
          </div>
          <div class="card hint">
            <h4>📷 Clear Focus</h4>
            <p>Make sure the leaves are sharp and in focus</p>
          </div>
          <div class="card hint">
            <h4>🔎 Show Details</h4>
            <p>Include spots, discoloration, or pests</p>
          </div>
        </div>
      </div>
    </section>

    <!-- SENSORS PAGE -->
    <section id="page-sensors" class="hidden">
      <div class="pageTitle">
        <h2>Sensor Readings</h2>
        <p>Real-time environmental data from your plant sensors</p>
      </div>

      <div class="grid">
        <div class="sensorHeader">
          <div class="badgeConn"><i class="fa-solid fa-wifi"></i> <span id="connText">Connected</span></div>
          <div class="subtle" id="lastUpdated">Last updated: --</div>
        </div>

        <div class="card" style="grid-column: 1 / -1; background: linear-gradient(90deg, #1bd49a, #18b07b); color:white; border:none;">
          <div style="display:flex; justify-content: space-between; align-items:center;">
            <div>
              <h3 style="margin:0 0 4px;">Plant Station Alpha</h3>
              <div style="opacity:.9; font-weight: 700;">Living Room • Monstera Deliciosa</div>
            </div>
            <div style="text-align:right; opacity:.95; font-weight: 800;">
              <div>Live Data</div>
              <div><span id="sensorsActive">--</span> sensors active • Battery <span id="batteryPct">--</span>%</div>
            </div>
          </div>
        </div>

        <div class="cards4">
          <div class="card gauge">
            <div class="icon" style="margin:0 auto;"><i class="fa-solid fa-temperature-high"></i></div>
            <div class="num"><span id="tempVal">--</span>°C</div>
            <h4>Temperature</h4>
            <div class="scale"><span>0°C</span><span>40°C</span></div>
          </div>

          <div class="card gauge">
            <div class="icon" style="margin:0 auto;"><i class="fa-solid fa-droplet"></i></div>
            <div class="num"><span id="humVal">--</span>%</div>
            <h4>Humidity</h4>
            <div class="scale"><span>0%</span><span>100%</span></div>
          </div>

          <div class="card gauge">
            <div class="icon" style="margin:0 auto;"><i class="fa-solid fa-seedling"></i></div>
            <div class="num"><span id="soilVal">--</span>%</div>
            <h4>Soil Moisture</h4>
            <div class="scale"><span>0%</span><span>100%</span></div>
          </div>

          <div class="card gauge">
            <div class="icon" style="margin:0 auto;"><i class="fa-solid fa-sun"></i></div>
            <div class="num"><span id="lightVal">--</span> lux</div>
            <h4>Light Level</h4>
            <div class="scale"><span>0</span><span>1000</span></div>
          </div>
        </div>

        <div class="card aqi">
          <div class="row">
            <div>
              <h3 style="margin:0 0 6px;">Air Quality Index</h3>
              <div class="subtle">Dynamic AQI score & CO₂ estimate</div>
            </div>
            <div class="badgeConn" style="background: rgba(255,255,255,0.18); border-color: rgba(255,255,255,0.28); color:white;">
              <i class="fa-solid fa-leaf"></i> <span id="aqiLabel">Good</span>
            </div>
          </div>

          <div class="aqiRow" style="margin-top: 12px;">
            <div class="aqiScore" id="aqiScore">--</div>
            <div class="aqiBar"><div class="aqiFill" id="aqiFill"></div></div>
            <div class="aqiMeta">CO₂: <span id="co2Val">--</span> ppm</div>
          </div>
        </div>

        <div class="card" style="grid-column: 1 / -1;">
          <h3 style="margin:0 0 6px;">Recommendations</h3>
          <div class="subtle" id="recText">--</div>
        </div>
      </div>
    </section>

    <!-- SEARCH PAGE -->
    <section id="page-search" class="hidden">
      <div class="grid">
        <div class="searchHeader">
          <h2>Search Plant Diseases</h2>
          <p>Find information about common diseases, symptoms, and treatments</p>
        </div>

        <div class="searchBox">
          <input id="searchInput" class="searchInput" placeholder="Search diseases, symptoms, or treatments..."/>
          <div class="filters">
            <span class="subtle" style="margin-right:8px;"><i class="fa-solid fa-filter"></i> Severity:</span>
            <button class="chip active" data-sev="all">All</button>
            <button class="chip" data-sev="low">Low</button>
            <button class="chip" data-sev="medium">Medium</button>
            <button class="chip" data-sev="high">High</button>
          </div>
        </div>

        <div class="popular" id="popularChips"></div>

        <div class="resultGrid" id="resultsGrid"></div>
      </div>
    </section>

  </main>
</div>

<!-- Charts -->
<script>
  const loadChartJS = () => new Promise(resolve => {
    const s = document.createElement("script");
    s.src = "https://cdn.jsdelivr.net/npm/chart.js@4.4.1/dist/chart.umd.min.js";
    s.onload = resolve;
    document.head.appendChild(s);
  });
</script>

<script>
  // ---------- Helpers ----------
  const rand = (min, max) => Math.random() * (max - min) + min;
  const rint = (min, max) => Math.floor(rand(min, max+1));
  const clamp = (v, a, b) => Math.max(a, Math.min(b, v));
  const nowTime = () => new Date().toLocaleTimeString([], {hour:'2-digit', minute:'2-digit', second:'2-digit'});

  function labelForScore(x){
    if (x >= 85) return {text:"Excellent", cls:"tag"};
    if (x >= 65) return {text:"Moderate", cls:"tag warn"};
    return {text:"Needs Care", cls:"tag bad"};
  }

  // ---------- Dynamic data model ----------
  function generateState(){
    // KPI-like counts
    const totalPlants = rint(8, 24);
    const avgHealth = rint(70, 96);
    const waterToday = rint(0, 5);

    // Scores 0..100
    const overall = clamp(avgHealth + rint(-8, 6), 55, 99);
    const hydration = clamp(rint(45, 95), 35, 98);
    const lightScore = clamp(rint(55, 100), 40, 100);

    // Alerts: depends on hydration & overall
    const alerts = (hydration < 55 ? rint(1, 3) : rint(0, 2));

    // Weekly trend
    const days = ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"];
    let base = rint(72, 90);
    const weekly = days.map((d, i) => clamp(base + rint(-6, 7) + Math.round(Math.sin(i/2)*4), 60, 100));

    // Donut distribution
    const healthy = clamp(Math.round(totalPlants * rand(0.55, 0.75)), 1, totalPlants);
    const moderate = clamp(Math.round(totalPlants * rand(0.15, 0.35)), 0, totalPlants-healthy);
    const needsCare = clamp(totalPlants - healthy - moderate, 0, totalPlants);

    // Environment lines
    const envMoist = days.map(() => clamp(rint(35, 75), 0, 100));
    const envTemp = days.map(() => clamp(rint(19, 32), 0, 45));

    // Activity
    const actions = [
      {icon:"fa-droplet", text:"Watered Monstera"},
      {icon:"fa-sun", text:"Light adjusted for Fern"},
      {icon:"fa-heart-pulse", text:"Health check completed"},
      {icon:"fa-seedling", text:"Fertilizer added"},
      {icon:"fa-bug", text:"Pest inspection performed"},
      {icon:"fa-flask", text:"Nutrient mix adjusted"},
    ];
    const activity = Array.from({length: 4}, () => {
      const a = actions[rint(0, actions.length-1)];
      const hours = rint(1, 36);
      return {...a, when: hours <= 12 ? `${hours} hours ago` : `${Math.ceil(hours/24)} days ago`};
    });

    // Sensors page data
    const temperature = Math.round(rand(18, 33));
    const humidity = rint(35, 80);
    const soil = rint(20, 75);
    const light = rint(150, 980);
    const aqi = rint(55, 95);
    const co2 = rint(380, 650);
    const battery = rint(55, 98);
    const sensorsActive = rint(3, 7);

    const recs = [];
    if (soil < 35) recs.push("Soil moisture is low. Consider watering within the next 2–3 hours.");
    if (temperature > 30) recs.push("Temperature is high. Provide shade or improve ventilation.");
    if (light < 280) recs.push("Light is low. Move the plant closer to a light source.");
    if (recs.length === 0) recs.push("Conditions are optimal. Keep monitoring for changes.");

    // Care schedule banner
    const tomorrow = rint(1, Math.min(5, totalPlants));
    const careText = `${tomorrow} plants need watering tomorrow morning`;

    // Notifications
    const notifCount = clamp(alerts + rint(0, 2), 0, 9);

    return {
      totalPlants, avgHealth, waterToday, alerts,
      overall, hydration, lightScore,
      days, weekly,
      healthy, moderate, needsCare,
      envMoist, envTemp,
      activity,
      sensors: {temperature, humidity, soil, light, aqi, co2, battery, sensorsActive},
      recText: recs.join(" "),
      careText,
      notifCount
    };
  }

  // ---------- Charts ----------
  let trendChart, donutChart, envChart;

  function initCharts(){
    const tctx = document.getElementById("trendChart");
    trendChart = new Chart(tctx, {
      type: "line",
      data: {
        labels: ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"],
        datasets: [{
          label: "Health",
          data: [80,82,84,83,86,88,85],
          borderColor: "#18b07b",
          backgroundColor: "rgba(24,176,123,0.12)",
          fill: true,
          tension: 0.35,
          pointRadius: 3
        }]
      },
      options: {
        responsive: true,
        plugins: { legend: { display:false } },
        scales: {
          y: { min: 60, max: 100, grid: { color:"rgba(15,23,42,0.06)" } },
          x: { grid: { display:false } }
        }
      }
    });

    const dctx = document.getElementById("donutChart");
    donutChart = new Chart(dctx, {
      type: "doughnut",
      data: {
        labels: ["Healthy","Moderate","Needs Care"],
        datasets: [{
          data: [8,3,1],
          backgroundColor: ["#18b07b","#f59e0b","#ef4444"],
          borderWidth: 0
        }]
      },
      options: {
        plugins: { legend: { position:"bottom", labels:{ boxWidth:12, usePointStyle:true } } },
        cutout: "70%"
      }
    });

    const ectx = document.getElementById("envChart");
    envChart = new Chart(ectx, {
      type: "line",
      data: {
        labels: ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"],
        datasets: [
          {
            label: "Moisture %",
            data: [55,52,60,57,54,62,58],
            borderColor: "#3b82f6",
            backgroundColor: "rgba(59,130,246,0.10)",
            fill: false,
            tension: 0.35,
            pointRadius: 2
          },
          {
            label: "Temp °C",
            data: [23,22,25,24,23,26,24],
            borderColor: "#f59e0b",
            backgroundColor: "rgba(245,158,11,0.10)",
            fill: false,
            tension: 0.35,
            pointRadius: 2
          }
        ]
      },
      options: {
        plugins: { legend: { position:"bottom", labels:{ boxWidth:12, usePointStyle:true } } },
        scales: {
          y: { grid: { color:"rgba(15,23,42,0.06)" } },
          x: { grid: { display:false } }
        }
      }
    });
  }

  // ---------- Render ----------
  function render(state){
    // KPIs
    document.getElementById("kpiPlants").textContent = state.totalPlants;
    document.getElementById("kpiHealth").textContent = state.avgHealth;
    document.getElementById("kpiWater").textContent = state.waterToday;
    document.getElementById("kpiAlerts").textContent = state.alerts;

    // Notifications badge
    document.getElementById("notifCount").textContent = state.notifCount;

    // Right stats (scores)
    const overallLbl = labelForScore(state.overall);
    const hydLbl = labelForScore(state.hydration);
    const lightLbl = labelForScore(state.lightScore);

    document.getElementById("overallScore").textContent = state.overall;
    document.getElementById("hydrationScore").textContent = state.hydration;
    document.getElementById("lightScore").textContent = state.lightScore;

    document.getElementById("barOverall").style.width = state.overall + "%";
    document.getElementById("barHydration").style.width = state.hydration + "%";
    document.getElementById("barLight").style.width = state.lightScore + "%";

    const tagOverall = document.getElementById("tagOverall");
    tagOverall.className = overallLbl.cls;
    tagOverall.innerHTML = `<i class="fa-solid ${state.overall>=85?'fa-circle-check':state.overall>=65?'fa-circle-exclamation':'fa-triangle-exclamation'}"></i> ${overallLbl.text}`;

    const tagHydration = document.getElementById("tagHydration");
    tagHydration.className = hydLbl.cls;
    tagHydration.innerHTML = `<i class="fa-solid ${state.hydration>=85?'fa-circle-check':state.hydration>=65?'fa-circle-exclamation':'fa-triangle-exclamation'}"></i> ${hydLbl.text}`;

    const tagLight = document.getElementById("tagLight");
    tagLight.className = lightLbl.cls;
    tagLight.innerHTML = `<i class="fa-solid ${state.lightScore>=85?'fa-circle-check':state.lightScore>=65?'fa-circle-exclamation':'fa-triangle-exclamation'}"></i> ${lightLbl.text}`;

    // Banner
    document.getElementById("careText").textContent = state.careText;

    // Activity list
    const ul = document.getElementById("activityList");
    ul.innerHTML = "";
    state.activity.forEach(a => {
      const li = document.createElement("li");
      li.innerHTML = `
        <i class="fa-solid ${a.icon}"></i>
        <div>
          <strong>${a.text}</strong>
          <small>${a.when}</small>
        </div>
      `;
      ul.appendChild(li);
    });

    // Charts update
    trendChart.data.labels = state.days;
    trendChart.data.datasets[0].data = state.weekly;
    trendChart.update();

    donutChart.data.datasets[0].data = [state.healthy, state.moderate, state.needsCare];
    donutChart.update();

    envChart.data.labels = state.days;
    envChart.data.datasets[0].data = state.envMoist;
    envChart.data.datasets[1].data = state.envTemp;
    envChart.update();

    // Sensors page values
    const s = state.sensors;
    document.getElementById("lastUpdated").textContent = "Last updated: " + nowTime();
    document.getElementById("sensorsActive").textContent = s.sensorsActive;
    document.getElementById("batteryPct").textContent = s.battery;

    document.getElementById("tempVal").textContent = s.temperature;
    document.getElementById("humVal").textContent = s.humidity;
    document.getElementById("soilVal").textContent = s.soil;
    document.getElementById("lightVal").textContent = s.light;

    // AQI
    document.getElementById("aqiScore").textContent = s.aqi;
    document.getElementById("co2Val").textContent = s.co2;
    document.getElementById("aqiFill").style.width = clamp(s.aqi, 0, 100) + "%";

    const aqiLabel = document.getElementById("aqiLabel");
    aqiLabel.textContent = s.aqi >= 80 ? "Good" : s.aqi >= 65 ? "Moderate" : "Poor";

    // Recommendation text
    document.getElementById("recText").textContent = state.recText;

    // System status text
    document.getElementById("sysText").textContent = state.alerts > 0 ? "Some warnings detected" : "All sensors online";
  }

  // ---------- Upload analysis (dynamic, no fixed output) ----------
  function analyzeImageDynamic(){
    // Fake analysis is allowed as prototype: output varies each time
    const issues = [
      {name:"Possible nutrient deficiency", sev:"Medium", tip:"Consider balanced fertilizer and check soil pH."},
      {name:"Possible fungal spots", sev:"High", tip:"Remove affected leaves and improve airflow."},
      {name:"Water stress detected", sev:"Medium", tip:"Adjust irrigation schedule and check soil moisture."},
      {name:"No critical issues detected", sev:"Low", tip:"Keep monitoring with sensors and weekly checks."},
      {name:"Light burn suspected", sev:"Low", tip:"Reduce direct sunlight exposure during peak hours."}
    ];
    const pick = issues[rint(0, issues.length-1)];
    return pick;
  }

  // ---------- Search database (dynamic content, filtered) ----------
  const diseaseDB = [
    {title:"Fungal Diseases", count: () => rint(25, 60), sev:"high", icon:"fa-bacterium"},
    {title:"Pest Infestations", count: () => rint(20, 45), sev:"medium", icon:"fa-bug"},
    {title:"Nutrient Deficiencies", count: () => rint(10, 35), sev:"low", icon:"fa-flask"},
    {title:"Root Rot", count: () => rint(8, 24), sev:"high", icon:"fa-skull-crossbones"},
    {title:"Mold", count: () => rint(6, 18), sev:"medium", icon:"fa-cloud"},
    {title:"Wilting", count: () => rint(12, 28), sev:"medium", icon:"fa-leaf"}
  ];

  const popular = ["Root rot","Yellow leaves","Brown spots","Wilting","Mold","Pests","Dry soil","Overwatering"];

  let currentSeverity = "all";

  function severityBadge(sev){
    if (sev === "low") return `<span class="badgeSeverity sevLow"><i class="fa-solid fa-circle-check"></i> Low</span>`;
    if (sev === "medium") return `<span class="badgeSeverity sevMed"><i class="fa-solid fa-circle-exclamation"></i> Medium</span>`;
    return `<span class="badgeSeverity sevHigh"><i class="fa-solid fa-triangle-exclamation"></i> High</span>`;
  }

  function renderSearch(){
    const q = document.getElementById("searchInput").value.trim().toLowerCase();
    const grid = document.getElementById("resultsGrid");
    grid.innerHTML = "";

    const items = diseaseDB
      .filter(d => currentSeverity === "all" ? true : d.sev === currentSeverity)
      .filter(d => q.length === 0 ? true : d.title.toLowerCase().includes(q) || popular.some(p => p.toLowerCase().includes(q)));

    // If query is symptom-like, produce dynamic "AI suggestion" card at top
    if(q.length >= 3){
      const aiGuess = (q.includes("yellow") ? "Nutrient deficiency likely" :
                       q.includes("spots") ? "Fungal infection likely" :
                       q.includes("dry") ? "Water stress likely" :
                       q.includes("mold") ? "Humidity issue likely" :
                       q.includes("pest") ? "Pest infestation likely" :
                       "General plant stress detected");
      const aiSev = (q.includes("spots") || q.includes("root") ? "high" : q.includes("dry") ? "medium" : "low");

      const card = document.createElement("div");
      card.className = "card resCard";
      card.innerHTML = `
        <div class="icon" style="background: rgba(59,130,246,0.10); border-color: rgba(59,130,246,0.18); color:#1d4ed8;">
          <i class="fa-solid fa-wand-magic-sparkles"></i>
        </div>
        <h3>AI Suggestion</h3>
        <p>${aiGuess}. Try checking sensor readings & upload a photo for deeper analysis.</p>
        ${severityBadge(aiSev)}
      `;
      grid.appendChild(card);
    }

    // category cards
    items.slice(0, 6).forEach(d => {
      const card = document.createElement("div");
      card.className = "card resCard";
      const entries = d.count();
      card.innerHTML = `
        <div class="icon" style="background: rgba(24,176,123,0.10); border-color: rgba(24,176,123,0.18); color:#0b8f67;">
          <i class="fa-solid ${d.icon}"></i>
        </div>
        <h3>${d.title}</h3>
        <p>${entries} entries</p>
        ${severityBadge(d.sev)}
      `;
      grid.appendChild(card);
    });

    if(items.length === 0){
      const empty = document.createElement("div");
      empty.className = "card";
      empty.style.gridColumn = "1 / -1";
      empty.innerHTML = `<h3 style="margin:0 0 6px;">No results</h3><div class="subtle">Try another keyword or change severity filter.</div>`;
      grid.appendChild(empty);
    }
  }

  function renderPopular(){
    const wrap = document.getElementById("popularChips");
    wrap.innerHTML = "";
    popular.forEach(p => {
      const btn = document.createElement("button");
      btn.className = "chip";
      btn.textContent = p;
      btn.addEventListener("click", () => {
        document.getElementById("searchInput").value = p;
        renderSearch();
      });
      wrap.appendChild(btn);
    });
  }

  // ---------- Navigation ----------
  const pages = ["dashboard","upload","sensors","search"];
  function showPage(name){
    pages.forEach(p => {
      document.getElementById("page-" + p).classList.toggle("hidden", p !== name);
    });

    document.querySelectorAll(".nav button").forEach(b => {
      b.classList.toggle("active", b.dataset.page === name);
    });

    const titleMap = {
      dashboard: "Dashboard",
      upload: "Upload Image",
      sensors: "Sensor Data",
      search: "Disease Search"
    };
    document.getElementById("crumbText").textContent = titleMap[name] || "Dashboard";
  }

  document.querySelectorAll(".nav button").forEach(btn => {
    btn.addEventListener("click", () => showPage(btn.dataset.page));
  });

  // ---------- Upload interactions ----------
  const dropZone = document.getElementById("dropZone");
  const fileInput = document.getElementById("fileInput");
  const previewImg = document.getElementById("previewImg");
  const analysisBox = document.getElementById("analysisBox");

  dropZone.addEventListener("click", () => fileInput.click());
  dropZone.addEventListener("dragover", (e) => { e.preventDefault(); dropZone.style.borderColor = "rgba(24,176,123,0.55)"; });
  dropZone.addEventListener("dragleave", () => { dropZone.style.borderColor = "rgba(15,23,42,0.12)"; });
  dropZone.addEventListener("drop", (e) => {
    e.preventDefault();
    dropZone.style.borderColor = "rgba(15,23,42,0.12)";
    const file = e.dataTransfer.files?.[0];
    if(file) handleFile(file);
  });

  fileInput.addEventListener("change", (e) => {
    const file = e.target.files?.[0];
    if(file) handleFile(file);
  });

  function handleFile(file){
    const url = URL.createObjectURL(file);
    previewImg.src = url;
    previewImg.style.display = "block";

    const a = analyzeImageDynamic();
    analysisBox.classList.remove("hidden");
    analysisBox.innerHTML = `
      <h3 style="margin:0 0 6px;">AI Analysis Result</h3>
      <div class="subtle">Filename: ${file.name} • Updated: ${nowTime()}</div>
      <div style="margin-top:12px; font-weight:900;">${a.name}</div>
      <div style="margin-top:8px;">Recommended action: <span class="subtle">${a.tip}</span></div>
      <div style="margin-top:10px;">${severityBadge(a.sev.toLowerCase())}</div>
    `;
  }

  // ---------- Buttons ----------
  document.getElementById("refreshBtn").addEventListener("click", () => {
    appState = generateState();
    render(appState);
    renderSearch();
  });

  document.getElementById("scheduleBtn").addEventListener("click", () => {
    alert("✅ Care schedule saved (prototype). Data will update dynamically.");
  });

  document.getElementById("viewScheduleBtn").addEventListener("click", () => {
    alert("📅 Next schedule opened (prototype).");
  });

  document.getElementById("notifBtn").addEventListener("click", () => {
    alert("🔔 Notifications (prototype): dynamic alerts based on live conditions.");
  });

  // ---------- Search filters ----------
  document.querySelectorAll(".chip[data-sev]").forEach(ch => {
    ch.addEventListener("click", () => {
      document.querySelectorAll(".chip[data-sev]").forEach(x => x.classList.remove("active"));
      ch.classList.add("active");
      currentSeverity = ch.dataset.sev;
      renderSearch();
    });
  });

  document.getElementById("searchInput").addEventListener("input", renderSearch);

  // ---------- Auto-update loop (dynamic, no fixed data) ----------
  let appState;
  function tick(){
    appState = generateState();
    render(appState);
    // keep search cards dynamic too
    renderSearch();

    // simulate connection fluctuation in sensors page
    const connected = Math.random() > 0.06;
    document.getElementById("connText").textContent = connected ? "Connected" : "Reconnecting...";
    document.getElementById("sysText").textContent = connected ? (appState.alerts>0 ? "Some warnings detected" : "All sensors online")
                                                             : "Connection unstable";

    // re-run
    setTimeout(tick, 6000);
  }

  // ---------- Init ----------
(async function startApp(){
  await loadChartJS();        // ⬅️ انتظري Chart.js
  initCharts();               // ⬅️ الآن ارسم
  renderPopular();
  showPage("dashboard");
  tick();
})();
</script>

</body>
</html>

"""
display(HTML(html_code))

